In [142]:
import pandas as pd
import networkx as nx
from datetime import timedelta
import re

In [143]:
def extraire_stop_id(stop_id):
    """
    Cette fonction extrait la partie pertinente du stop_id.
    Elle supprime le préfixe 'StopPoint:' et la partie supplémentaire après le premier '-'.
    """
    # Si le stop_id commence par 'StopPoint:', on le retire
    if stop_id.startswith('StopPoint:'):
        stop_id = stop_id[len('StopPoint:'):]
    
    # Extraire la partie avant le premier '-'
    stop_id = stop_id.split('-')[0]
    
    return stop_id

In [144]:
def charger_gtfs(fichiers):
    """
    Charge les fichiers GTFS nécessaires.
    :param fichiers: Dictionnaire contenant les chemins vers les fichiers GTFS.
    :return: DataFrames des stops, trips, stop_times.
    """
    stops = pd.read_csv(fichiers["stops"])
    stop_times = pd.read_csv(fichiers["stop_times"])
    trips = pd.read_csv(fichiers["trips"])
    routes = pd.read_csv(fichiers["routes"])
    
    stop_times["stop_id"] = stop_times["stop_id"].str.replace("StopArea:", "", regex=False)

    return stops, stop_times, trips, routes

In [145]:
def corriger_heure_gtfs(heure):
    """
    Corrige les heures GTFS dépassant 23:59:59 (par ex. 24:20:00).
    :param heure: Chaîne horaire au format HH:MM:SS.
    :return: Heure corrigée sous forme de timedelta.
    """
    try:
        # Séparer les heures, minutes et secondes
        h, m, s = map(int, heure.split(":"))
        # Convertir en timedelta
        return timedelta(hours=h, minutes=m, seconds=s)
    except ValueError:
        raise ValueError(f"Heure invalide dans les données : {heure}")


In [146]:
def construire_graphe(stops, stop_times):
    """
    Construit un graphe à partir des données GTFS.
    :param stops: DataFrame des arrêts (stops.txt).
    :param stop_times: DataFrame des horaires (stop_times.txt).
    :return: Un graphe orienté pondéré.
    """
    G = nx.DiGraph()

    # Nettoyer les stop_id dans les nœuds du graphe
    stop_times["departure_time"] = stop_times["departure_time"].apply(corriger_heure_gtfs)
    stop_times["arrival_time"] = stop_times["arrival_time"].apply(corriger_heure_gtfs)

    stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])

    # Ajouter des nœuds au graphe en utilisant les stop_id
    for stop_id in stops["stop_id"]:
        stop_id_nettoye = extraire_stop_id(stop_id)  # Appliquer le nettoyage
        G.add_node(stop_id_nettoye)  # Ajouter le nœud nettoyé dans le graphe

    # Ajouter des arêtes au graphe
    for trip_id, group in stop_times.groupby("trip_id"):
        group = group.reset_index()
        for i in range(len(group) - 1):
            origine = extraire_stop_id(group.loc[i, "stop_id"])  # Appliquer le nettoyage
            destination = extraire_stop_id(group.loc[i + 1, "stop_id"])  # Appliquer le nettoyage
            temps_depart = group.loc[i, "departure_time"]
            temps_arrivee = group.loc[i + 1, "arrival_time"]
            poids = (temps_arrivee - temps_depart).total_seconds() // 60  # Temps en minutes

            if poids >= 0:  # On évite les poids négatifs
                G.add_edge(origine, destination, weight=poids)

    return G


In [147]:
def trouver_chemin_optimal(graphe, depart, arrivee):
    """
    Trouve le chemin optimal entre deux arrêts.
    :param graphe: Graphe orienté pondéré.
    :param depart: ID de l'arrêt de départ.
    :param arrivee: ID de l'arrêt d'arrivée.
    :return: Chemin optimal et durée totale.
    """
    try:
        chemin = nx.shortest_path(graphe, source=depart, target=arrivee, weight="weight")
        duree = nx.shortest_path_length(graphe, source=depart, target=arrivee, weight="weight")
        return chemin, duree
    except nx.NetworkXNoPath:
        return None, float('inf')


In [148]:
fichiers = {
    "stops": "assets/gtfs/stops.txt",
    "stop_times": "assets/gtfs/stop_times.txt",
    "trips": "assets/gtfs/trips.txt",
    "routes": "assets/gtfs/routes.txt"
}

In [149]:
stops, stop_times, trips, routes = charger_gtfs(fichiers)

In [150]:
stops["stop_id"] = stops["stop_id"].apply(extraire_stop_id)

In [151]:
graphe = construire_graphe(stops, stop_times)

In [152]:
def trouver_stop_id(stops, nom_gare):
    """
    Trouve le stop_id d'une gare par son nom.
    :param stops: DataFrame des arrêts.
    :param nom_gare: Nom (ou partie du nom) de la gare.
    :return: Liste des stop_id correspondants.
    """
    resultats = stops[stops["stop_name"].str.contains(nom_gare, case=False)]
    return resultats[["stop_id", "stop_name"]]

In [153]:
depart_ids = trouver_stop_id(stops, "Paris")
end_ids = trouver_stop_id(stops, "Strasbourg")

In [154]:
depart = depart_ids.iloc[0]["stop_id"]
arrivee = end_ids.iloc[0]["stop_id"]

In [155]:
if depart in graphe.nodes and arrivee in graphe.nodes:
    print(f"Le départ ({depart}) et l'arrivée ({arrivee}) sont dans le graphe.")
else:
    print(f"Erreur : L'un des arrêts ({depart} ou {arrivee}) n'est pas dans le graphe.")


Le départ (OCE87113001) et l'arrivée (OCE87212027) sont dans le graphe.


In [156]:
if depart in graphe.nodes:
    voisins_depart = list(graphe.neighbors(depart))
    print(f"Voisins du départ ({depart}) : {voisins_depart}")


Voisins du départ (OCE87113001) : []
